In [12]:
import os
import sys
import re
from operator import itemgetter
from datetime import datetime
from utils import split_text_at_punctuation

from langchain.document_loaders.text import TextLoader

from langchain_community.document_loaders import DirectoryLoader
from langchain_community.embeddings import HuggingFaceEmbeddings

from langchain.chat_models.openai import ChatOpenAI

from langchain_core.prompts import ChatPromptTemplate

from langchain_core.output_parsers import PydanticOutputParser

from langchain.chains.combine_documents import create_stuff_documents_chain

from langchain.pydantic_v1 import BaseModel, Field

from langchain.vectorstores import qdrant

from config import GPT_API

import re
from typing import Annotated, List, Dict, TypedDict
from langgraph.graph.message import add_messages, AnyMessage
from utils import check_latex_safety, validate_words



os.environ["OPENAI_API_KEY"] = GPT_API

current_date = datetime.now().strftime('%B%Y')



In [13]:
SET_TEMPERATURE = 0.2
MODEL = "gpt-4o-2024-05-13"#"gpt-3.5-turbo"

LLM_MODEL = ChatOpenAI(
    model=MODEL,
    name="Agent for job applications",
    temperature=SET_TEMPERATURE,
    n=1,
    )

In [14]:
# loader = TextLoader("ai_engineer_software_engineer.txt")

loader = DirectoryLoader(path="jobtemplates/")

load_applications = loader.load()

embedding_model = HuggingFaceEmbeddings(
    model_name="sentence-transformers/all-mpnet-base-v2",
    encode_kwargs = {'normalize_embeddings':False}
)

vectorstore = qdrant.Qdrant.from_documents(
    documents=load_applications,
    embedding=embedding_model,
    location=":memory:"
)

In [15]:
skills = [
    "Business analytics",
    "Business maturity",
    "Strategy",
    "Non-technical and technical communication",
    "Algorithms & datastrucures",
    "Software Engineering",
    "detail oriented",
    "Creative thinker",
    "Problem solving",
    "Critical thinking",
    "Team player",
    "Time management",
    "Adaptability",
    "Conflict resolution",
    "Collaborative",
    "Dilligent"
]

IT_Management = [
    "ITIL",
    "SAFe",
    "PRINCE2",
    "CMMI",
    "SCRUM",
    "Agile development",
    "UML(frequency, class or C4)",
    "Stakeholder classification"
]

Programming_languages = [
    "Python intermediate level",
    "SQL working understanding",
    "R working understanding",
    "JavaScript working understanding"
]

technical_skills = [
    "git"
    "Statistical modelling",
    "Fundamental Azure knowledge",
    "PostGres",
    "Neo4J",
    "Qdrant",
    "ANNOY",
    "Docker",
    "scraping",
    "crawling",
    "MT5",
    "Bert",
    "FinBert",
    "T5",
    "Scrapy",
    "Numpy",
    "Polars",
    "Pandas",
    "FastAPI",
    "VUE3",
    "TensorFlow2",
    "Hyggingface",
    "Pytorch",
    "SonarCube",
    "Seaborn(/matplotlib/Plotly)",
    "PyTest",
    "SKlearn"
    "Unsupervised learning: dimensionality reduction, explorative factor analysis, K-mean..",
    "Supervised learning: Random Forests, multiple logistic regression, SVP, NNs, Classification"
]

skills_dict = {
    'soft skills': skills,
    'IT Management': IT_Management,
    'Programming languages': Programming_languages,
    'Technical skills': technical_skills
}

do_not_use_words = [
    "abreast",
    "ardent",
    "cruisal",
    "deeply",
    "eagerly",
    "endeavors",
    "enhance",
    "enhanced",
    "enhancing",
    "extensive",
    "extensively", 
    "expert",
    "expertise",
    "facets"
    "forefront",
    "fostering",
    "fueled",
    "fulfilling",
    "honed",
    "intricacies",
    "intricate",
    "meticulous ",
    "perfect",
    "perfectly",
    "prowess",
    "profoundly",
    "realm",
    "seamlessly",
    "specialist",
    "stems",
    "thrilled",
    "versed"
]


In [16]:
query_for_search = """


Engineering

In the platform area in Digital Architecture, Data and AI we lay the groundwork for other product teams to delivering accurate, available, and comprehensive data products to enable actionable insights. In addition to this we create trusted and curated enterprise data products that span the entire business of Vestas. This newly established department is an integral component of Vestas' innovative Digital Powerhouse. Our mission is to empower customers, partners, and colleagues to seamlessly discover, access, and connect essential information for informed decisions and impactful actions.

 
Digital Solutions & Development > Digital Solutions > Chapter - Data Engineering & Architecture

  
As a Data Engineer in the platform area, you will collaborate closely with colleagues both inside and outside of the platform area.
 
Your role involves leveraging a range of cloud technologies and tools tailored to the specific product you are working on. This encompasses working with technologies and tools such as Snowflake, databricks, dbt (data build tool) and Azure Services (storage accounts, key vaults).

 

Responsibilities

Your key responsibilities will be:
 

Designing, constructing, and maintaining scalable, reliable, and efficient data products
Managing and monitoring data products
Engaging in close collaboration with stakeholders who consume the data product
Establishing necessary integrations with various sources to enable data extraction
 
 
Qualifications 

Degree in Business Intelligence, Data Engineering or Software Development and/or 3 to 5 years of Professional work experience in a similar field
Comfortable in a dynamic and changeable working day
Comprehensive analytical and problem-solving skills
Advanced communication and collaboration skills, with the ability to work effectively in a cross-functional team environment
Proficient communication skills in English (our corporate language), Danish is not required
 
Competencies

 We envision that you possess experience in designing, building, and maintaining data transformation logic. Furthermore, you may see yourself reflected in any of the following categories:

Proficiency in programming languages like SQL and Python - experience with dbt, spark, Airflow or Terraform is considered an asset
Experience in the development code in a team using devops techniques and agile development methodologies
Expertise in working with various Data Warehouse solutions and constructing data products using technologies such as Snowflake, Databricks, Azure Data Engineering Stack (like storage accounts, key vaults, Synapse, MSSQL, etc.)
Understanding of data warehouse modelling methodologies (Kimball, data vault et al.) as well as concepts like data mesh or similar
 
On a personal level, we anticipate that you:

Possess a collaborative and open-minded nature, eager to contribute to a globally diverse cross-functional team within the organization
Display curiosity and motivation for developing innovative data products that generate value exhibing positive communication skills, coupled with a positive, problem-solving approach to accomplishing tasks
Thrive in diverse environments and exhibit flexibility in adapting to evolving conditions embracing a commitment to continuous learning and a desire to contribute to the collective growth of the team
 
What we offer 

You will join a newly established, innovative, and committed team committed to support the business through the development of enterprise data products. You will also have the possibility to be part of forming how to best build data products. You will experience an environment that promotes continuous learning, enabling you to actualize your ambitions and get the chance to work in an agile office environment. While we hold our team members to high individual standards of collaboration, accountability, and meeting deadlines, we provide unwavering support to one another, collectively celebrating successes and addressing challenges.


"""

semilarity_document_template = vectorstore.similarity_search_with_score(
    query = query_for_search,
    k = 1,
    score_threshold=0.1)

print("semilarity_document_template", semilarity_document_template)

semilarity_document_template [(Document(page_content="Data Scientist\n\nI reached out due to my interest in AI, Software Development/Engineering, and Knowledge management.\n\nMotivation I am driven to progress in the field of AI, particularly focusing on Large Language Models, knowledge extraction, and Natural Language Processing. My goal is to ensure that every AI implementation has a solid foundation of reliable data. I am dedicated to setting up thorough data agreements to back these efforts and am excited about the prospect of working alongside your team. My passion for AI is motivated by genuine curiosity and the satisfaction derived from tackling complex problems. Whether working independently or as part of a team, I am committed to addressing every necessary element to deliver a well-rounded AI solution, including aspects of security and compliance with GDPR and AI-ACT. I am enthusiastic about engaging in an environment that encourages detailed and strategic discussions on AI ap

In [17]:
VACANCY_ANALYSIS_PROMT = ChatPromptTemplate.from_messages(
    [
        ('system',"""
                You are an assisatant to a human resource manager//
                You are to assist in the analysis of a job vacancy//
                Identify vocal points of interest that the company is looking for//
                Identify the company name//
                Identify the job title//
                Identify the skills and technical experience required for the job vacancy provided here to be stored as a dictionary employee skill requirement//
                
        """),

        ('human',"""
                Given the job vacancy, you are to analyse the following in detail: {SomeVacantPosition}//
                Use these skills {my_skills} to conduct an analysis between job requirements and find matching skills//
                Output should contain a list of matching skills required for the job vacancy//
                {format_instructions_1}
        """),
    ]
)

## Output data structure
class OutputStuctureV1(BaseModel):
    company_name: str = Field(description="identified company name")
    job_title: str = Field(description="identified job title")
    analysis_output: str = Field(description="analysis of the job vacancy")
    employees_skills_requirement: dict = Field(description="identified skills and technical experience required for the job vacancy")
    matching_skills: dict = Field(description="matching skills in the job vacancy")

parser_1 = PydanticOutputParser(pydantic_object=OutputStuctureV1)

format_messages = VACANCY_ANALYSIS_PROMT.format(
    SomeVacantPosition = query_for_search,
    my_skills = skills_dict,
    format_instructions_1 = parser_1.get_format_instructions())


chain = LLM_MODEL | parser_1 

analysis_chain = chain.invoke(format_messages)


In [18]:
# Extracting identified information from analysis_chain
identified_company_name = analysis_chain.company_name
identified_job_title = analysis_chain.job_title
identified_skill_requirements = analysis_chain.employees_skills_requirement
identified_matching_skills = analysis_chain.matching_skills
identified_analysis_output = analysis_chain.analysis_output

# Create the output dictionary according to the schema
output = {
    "company_name": identified_company_name,
    "job_title": identified_job_title,
    "analysis_output": identified_analysis_output,
    "employees_skills_requirement": identified_skill_requirements
}

# Getting keys from the dictionary
get_employee_requirements_keys = identified_skill_requirements.keys()

# Create an itemgetter object with these keys
get_employee_requirements_lists = itemgetter(*get_employee_requirements_keys)

# Applying itemgetter to the dictionary to get the lists
employee_requirements = get_employee_requirements_lists(identified_skill_requirements)

# Zipping keys with their corresponding lists
lists_with_employee_requirements = zip(get_employee_requirements_keys, employee_requirements)

# If you need to process lists_with_employee_requirements further, you can do so here
get_matching_skills_keys = identified_matching_skills.keys()
get_matching_skills_lists = itemgetter(*get_matching_skills_keys)
matching_skills = get_matching_skills_lists(identified_matching_skills)
lists_with_matching_skills = zip(get_matching_skills_keys, matching_skills)
# Example usage:
# print the output dictionary
print(list(lists_with_employee_requirements))
print(list(lists_with_matching_skills))





[('soft_skills', ['Comprehensive analytical and problem-solving skills', 'Advanced communication and collaboration skills', 'Proficient communication skills in English', 'Collaborative and open-minded nature', 'Positive communication skills', 'Problem-solving approach', 'Flexibility in adapting to evolving conditions', 'Commitment to continuous learning']), ('technical_skills', ['Proficiency in programming languages like SQL and Python', 'Experience with dbt, Spark, Airflow, or Terraform', 'Experience in development code using devops techniques and agile methodologies', 'Expertise in working with Data Warehouse solutions like Snowflake, Databricks, Azure Data Engineering Stack', 'Understanding of data warehouse modelling methodologies (Kimball, data vault, etc.)', 'Concepts like data mesh'])]
[('soft_skills', ['Problem solving', 'Critical thinking', 'Team player', 'Adaptability', 'Collaborative']), ('technical_skills', ['Python intermediate level', 'SQL working understanding', 'Agile d

In [19]:
TEXT_GENERATION_PROMPT = ChatPromptTemplate.from_messages(
    [
        ('system',"""
         You are to assist in setting up a job application template//
         The total amount of characters that can be used is 4000, include white spaces//
         
         Grammatical correctness is essential//
         Use casual business language//
         Ensure, the English language is equal to EITLS c1 score//
         The template job application must be in English//
         200-300 characters for the introduction section//
         800-1000 characters for the motivation section//
         500-800 characters for the skills section//
         560 characters for the masters section//
         390 characters for the bachelors section//
         300 characters for the continued learning section//
         200 characters for the thank you note//
         This template is the jobtemplate: {semilarity_jobtemplate}// 
         
        """),

        ('human',"""
         
         I have the following knowledge and skills which can be found in the following dictionary {skills}//
         
         write two lines to generate a short introduction with interest in IT and AI with inspiration from the {analysis_output}//
         
         write motivation with matching pairs {skill_match} and {employee_requirements} and how these can be utilized for the company's benefi//
         
         write a section about skills somme of the skills and how they can be utilized for the company's benefit//

         keep educational background for later access and save the section about masters degree into latex_edu_master and the section about bachelors into latex_edu_bachelor//
         keep continued learning section and provide short context that I am willing to learn what is necessary for the company and specific role//
        
         write a short and consice thank you note to setup a cofee//

         I DO NOT have prior experience in a professional environment in programming, ONLY academia//
         I DO have prior experience in project management//
         {format_instructions_2}
        """),
    ]
)

## Output data structure
class OutputStuctureV2(BaseModel):
    
    latex_company_name: str = Field(description="Company name")
    latex_job_title: str = Field(description="Job title")
    latex_introduction: str = Field(description="Introduction")
    latex_motivation: str = Field(description="Motivation")
    latex_skills: str = Field(description="Skills")
    latex_edu_masters: str = Field(description="Masters")
    latex_edu_bachelor: str = Field(description="Bachelor")
    latex_continued_learning: str = Field(description="Continued learning")
    latex_thank_you: str = Field(description="Thank you for your time")


parser_2 = PydanticOutputParser(pydantic_object=OutputStuctureV2)

format_messages_2 = TEXT_GENERATION_PROMPT.format(
    analysis_output = identified_analysis_output,
    employee_requirements = lists_with_employee_requirements,
    skill_match = lists_with_matching_skills,
    skills = skills_dict,
    semilarity_jobtemplate = semilarity_document_template,
    format_instructions_2 = parser_2.get_format_instructions())

print(format_messages_2)
chain_2 = LLM_MODEL | parser_2

analysis_chain_2 = chain_2.invoke(format_messages_2)

System: 
         You are to assist in setting up a job application template//
         The total amount of characters that can be used is 4000, include white spaces//
         
         Grammatical correctness is essential//
         Use casual business language//
         Ensure, the English language is equal to EITLS c1 score//
         The template job application must be in English//
         200-300 characters for the introduction section//
         800-1000 characters for the motivation section//
         500-800 characters for the skills section//
         560 characters for the masters section//
         390 characters for the bachelors section//
         300 characters for the continued learning section//
         200 characters for the thank you note//
         This template is the jobtemplate: [(Document(page_content="Data Scientist\n\nI reached out due to my interest in AI, Software Development/Engineering, and Knowledge management.\n\nMotivation I am driven to progress in

In [20]:
# REMOVE_WORDS_PROMPT = ChatPromptTemplate.from_messages(
#     [
#         ('system',"""
         
#          keep the original text here {analysis_chain_2} and substitube words if they appear on this list {forbidden_words} with something similar //
         
#         {format_instructions_3}
         

#         """),
#     ]
# )

# parser_3 = PydanticOutputParser(pydantic_object=OutputStuctureV2)

# format_messages_3 = REMOVE_WORDS_PROMPT.format(
#     analysis_chain_2 = analysis_chain_2,
#     forbidden_words = do_not_use_words,
#     format_instructions_3 = parser_3.get_format_instructions())

# print(format_messages_3)

# analysis_chain_3 = chain_3.invoke(format_messages_3)


In [22]:
print(str(analysis_chain_2))

latex_company_name='Vestas' latex_job_title='Data Engineer' latex_introduction="I am excited to apply for the Data Engineer position at Vestas. My passion for IT and AI, combined with my academic background, aligns perfectly with the role's requirements of designing, constructing, and maintaining scalable, reliable, and efficient data products." latex_motivation="I am driven by the opportunity to work with cutting-edge cloud technologies such as Snowflake, Databricks, dbt, and Azure Services. My academic experience in data engineering and software development has equipped me with a solid foundation in SQL and Python, which I am eager to apply in a professional setting. I am particularly motivated by the prospect of collaborating with stakeholders to manage and integrate data products, ensuring they meet the highest standards of reliability and efficiency. My strong analytical and problem-solving skills, honed through various academic projects, will enable me to contribute effectively t

In [42]:
max_iterations = 3

class GraphState(TypedDict):
    """
    TypedDict for the graph state.

    Args:
        TypedDict: Base class for TypedDict.
    """
    error: str
    messages: Annotated[List[AnyMessage], add_messages]
    generation: str
    iterations: int

def generate_application(state: GraphState) -> GraphState:
    """
    Generate a job application based on the job template provided.

    Args:
        state (GraphState): The current graph state.

    Returns:
        state (dict): New key added to state, generation.
    """
    print("------ Generating application ------")

    # State
    messages = state['messages']
    iterations = state['iterations']
    error = state['error']


   

    # Generate solution
    APPLICATION_OUTPUT_PROMPT = ChatPromptTemplate.from_messages(
        [
            ('system',"""
            Validate, that the generated application is not using any of these words found here {forbidden_words} in {general_analysis}//
            {format_instructions_3}
            """
            )
        ]
    )

    parser_3 = PydanticOutputParser(pydantic_object=OutputStuctureV2)

    formatted_messages = APPLICATION_OUTPUT_PROMPT.format(
    general_analysis = analysis_chain_2,
    forbidden_words = do_not_use_words,
    format_instructions_3 = parser_3.get_format_instructions())

    messages += [(role, content) for role, content in formatted_messages]


    chain_3 = LLM_MODEL | parser_3

    application_solution = chain_3.invoke(messages)
    
    # Increment iterations
    iterations = iterations + 1

    return {
        "generation": application_solution,
        "messages": messages,
        "iterations": iterations,
        "error": error
    }
    
def check_generation(state: GraphState) -> GraphState:
    """
    Check the generated application for errors.

    Args:
        state (GraphState): The current graph state.

    Returns:
        state (GraphState): Updated graph state with error status.
    """

    print("------ Checking application ------")
    # State
    messages = state['messages']
    iterations = state['iterations']
    error = state['error']
    application_solution = state['generation']
    
    # Access solution components
    try:
        company_name = application_solution.latex_company_name
        job_title = application_solution.latex_job_title
        introduction = application_solution.latex_introduction
        motivation = application_solution.latex_motivation
        skills = application_solution.latex_skills
        masters = application_solution.latex_edu_masters
        bachelors = application_solution.latex_edu_bachelor
        continued_learning = application_solution.latex_continued_learning
        thank_you = application_solution.latex_thank_you

        # Validate words
        print('pre IF statement for validate words')
        forbidden_words_used = validate_words(do_not_use_words, company_name, job_title, introduction, motivation, skills, continued_learning, thank_you)
        if forbidden_words_used:
            print('inside IT statement forbidden words used:')
            raise ValueError(f"Forbidden words used: {', '.join(forbidden_words_used)}")

        # # Check LaTeX safety
        # safe_texts = check_latex_safety(company_name, job_title, introduction, motivation, skills, continued_learning, thank_you)
        # if any(text != original for text, original in zip(safe_texts, [company_name, job_title, introduction, motivation, skills, continued_learning, thank_you])):
        #     raise ValueError("LaTeX safety issues found in the generated application")

    except Exception as e:
        print("---APPLICATION CHECK: FAILED---")
        error_message = [(
            "user", f"Your generated application failed with: {e} because of {forbidden_words_used}. Reflect and review on this error and prior attempts to solve the issue. #1 state what you think went wrong and #2 try and solve this problem again. Return full solution. Use the output structure with company_name, job_title, introduction, motivation, skills, masters, bachelors, continued_learning, thank_you.")]
        messages += error_message
        error = "yes"
    else:
        print("---NO APPLICATION ERRORS---")
        error = "no"
    
    return {
        "generation": application_solution,
        "messages": messages,
        "iterations": iterations,
        "error": error
    }


def decide_to_finish(state: GraphState) -> str:
    """
    Determines whether to finish or retry based on the error status and iteration count.

    Args:
        state (GraphState): The current graph state.

    Returns:
        str: Next node to call.
    """
    error = state["error"]
    iterations = state["iterations"]

    if error == "no" or iterations >= max_iterations:
        print("---DECISION: FINISH---")
        return "end"
    else:
        print("---DECISION: RE-TRY SOLUTION---")
        return "generate"

#### Utilities
import uuid 

def _print_event(event: dict, _printed: set, max_length=1500):
    current_state = event.get("dialog_state")
    if current_state:
        print(f"Currently in: {current_state[-1]}")

    messages = event.get("messages", [])
    for role, content in messages:
        message_id = f"{role}_{hash(content)}"
        if message_id not in _printed:
            msg_repr = content if len(content) <= max_length else content[:max_length] + " ... (truncated)"
            print(f"{role}: {msg_repr}")
            _printed.add(message_id)


In [24]:
from langgraph.checkpoint.sqlite import SqliteSaver
from langgraph.graph import END, StateGraph

builder = StateGraph(GraphState)

# Define the nodes
builder.add_node("generate", generate_application)  # generation solution
builder.add_node("check_generation", check_generation)  # check generation

# Build graph
builder.set_entry_point("generate")
builder.add_edge("generate", "check_generation")
builder.add_conditional_edges(
    "check_generation",
    decide_to_finish,
    {
        "end": END,
        "generate": "generate",
    },
)

memory = SqliteSaver.from_conn_string(":memory:")
graph = builder.compile(checkpointer=memory)

In [45]:
import uuid
_printed = set()
thread_id = str(uuid.uuid4())
config = {
    "configurable": {
        # Checkpoints are accessed by thread_id
        "thread_id": thread_id,
    }
}

question = query_for_search
events = graph.stream(
    {"messages": [('user', question)], "iterations": 0}, config, stream_mode="values"
)
for event in events:
    _print_event(event, _printed)
    print("----")   
event['generation']

In [ ]:



def validate_words(do_not_use_words, *args):
    false_count = 0
    true_count = 0

    forbidden_words_used = []

    for arg in args:
        words = arg.split()
        for word in words:
            if word in do_not_use_words:
                false_count += 1
                forbidden_words_used.append(word)
            else:
                true_count += 1
    
    # Return a tuple with the counts and the list of forbidden words used
    return true_count, false_count, forbidden_words_used

true_count, false_count, forbidden_words_used = validate_words(
    do_not_use_words,
    final_company_name, final_jobtitle, final_introduction, final_motivation, 
    final_skills, final_masters, final_bachelors,  final_continued_learning, 
    final_thank_you
)

# Print the results
print(f"Number of allowed words: {true_count}")
print(f"Number of forbidden words used: {false_count}")
print(f"Forbidden words used: {forbidden_words_used}")

Number of allowed words: 459
Number of forbidden words used: 2
Forbidden words used: ['seamlessly', 'enhancing']


In [ ]:
#some regex to remove characters that intervene with latex commands
def check_latex_safety(*args):
    # Dictionary to map LaTeX special characters to their safe equivalents
    replacements = {
        '\\': ' ',          # backslash to space
        '{': ' ',           # curly brace to space
        '}': ' ',           # curly brace to space
        '#': ' ',           # hash to space
        '%': ' ',           # percent to space
        '&': 'and',         # ampersand to 'and'
        '_': ' ',           # underscore to space
        '^': ' ',           # caret to space
        '~': ' ',           # tilde to space
        '$': 'dollars',     # dollar to space
        '/': ' ',           # slash to space
        '*': ' ',           # asterisk to space
        '-': ' '            # hyphen to space
    }
    
    # Regex pattern to match any LaTeX special character
    pattern = r'[\\{}#%&_^\~$\/\*\-]'
    
    true_count = 0
    false_count = 0

    # Function to replace matched characters
    def replace_match(match):
        return replacements[match.group(0)]
    
    # Process each input text
    results = []
    for text in args:
        if re.search(pattern, text):
            false_count += 1
            safe_text = re.sub(pattern, replace_match, text)
        else:
            true_count += 1
            safe_text = text
        results.append(safe_text)

    print(f"Number of safe texts: {true_count}")
    print(f"Number of modified texts: {false_count}")

    return results


# Example usage:

(final_company_name, 
 final_jobtitle, 
 final_introduction, 
 final_motivation, 
 final_skills, 
 final_masters, 
 final_bachelors, 
 final_continued_learning, 
 final_thank_you) = check_latex_safety(
                                        final_company_name, 
                                        final_jobtitle, 
                                        final_introduction, 
                                        final_motivation, 
                                        final_skills, 
                                        final_masters, 
                                        final_bachelors, 
                                        final_continued_learning, 
                                        final_thank_you)

# Directory where the variables.tex file will be saved
directory = 'companies_applied_for'

# Create the directory if it does not exist
if not os.path.exists(directory):
    os.makedirs(directory)

company_directory = os.path.join(directory, final_company_name)
if not os.path.exists(company_directory):
    os.makedirs(company_directory)

# Write these variables to a .tex file in the specified directory
resume_file_path = os.path.join(company_directory, final_company_name + '.tex')
with open(resume_file_path, 'w') as text_for_latex:
    text_for_latex.write(f"\\newcommand{{\\finalCompanyName}}{{{final_company_name}}}\n")
    text_for_latex.write(f"\\newcommand{{\\finalJobtitle}}{{{final_jobtitle}}}\n")
    text_for_latex.write(f"\\newcommand{{\\finalIntroduction}}{{{final_introduction}}}\n")
    text_for_latex.write(f"\\newcommand{{\\finalMotivation}}{{{final_motivation}}}\n")
    text_for_latex.write(f"\\newcommand{{\\finalSkills}}{{{final_skills}}}\n")
    text_for_latex.write(f"\\newcommand{{\\finalEducationMaster}}{{{final_masters}}}\n")
    text_for_latex.write(f"\\newcommand{{\\finalEducationBachelor}}{{{final_bachelors}}}\n")
    text_for_latex.write(f"\\newcommand{{\\finalContinuedLearning}}{{{final_continued_learning}}}\n")
    text_for_latex.write(f"\\newcommand{{\\finalThankYou}}{{{final_thank_you}}}\n")

latex_filename = f'JMangabat_{final_company_name}_{final_jobtitle}_{current_date}.tex'

latex_file_path = os.path.join(company_directory, latex_filename)

# Check if the file already exists and create a unique file name if it does
file_counter = 1
while os.path.exists(latex_file_path):
    new_file_name = f'JMangabat_{final_company_name}_{final_jobtitle}_{current_date}_{file_counter}.tex'
    latex_file_path = os.path.join(company_directory, new_file_name)
    file_counter += 1

template_tex = "main_setup.tex"

tex_content = tex_content = f"""
%----------------------------------------------------------------------------------------
% PACKAGES AND OTHER DOCUMENT CONFIGURATIONS
%----------------------------------------------------------------------------------------

\\documentclass{{article}}
% \\usepackage{{charter}} % Use the Charter font
\\usepackage{{graphicx}} % Required for including images
\\usepackage{{fancyhdr}} % Required for customizing headers and footers
\\usepackage{{setspace}} % Remove paragraph indentation
\\usepackage{{titlesec}} % Used to customize the \\section command
\\usepackage[
    a4paper, % Paper size
    top=15mm, % Top margin
    bottom=15mm, % Bottom margin
    left=15mm, % Left margin
    right=15mm, % Right margin
    % showframe % Uncomment to show frames around the margins for debugging purposes
]{{geometry}}

% \\setlength{{\\parindent}}{{0pt}} % Paragraph indentation
\\setlength{{\\parskip}}{{-0.7em}} % Vertical space between paragraphs

\\fancypagestyle{{firstpage}}{{%
    \\fancyhf{{}} % Clear default headers/footers
    \\renewcommand{{\\headrulewidth}}{{0pt}} % No header rule
    \\renewcommand{{\\footrulewidth}}{{1pt}} % Footer rule thickness
}}

\\fancypagestyle{{subsequentpages}}{{%
    \\fancyhf{{}} % Clear default headers/footers
    \\renewcommand{{\\headrulewidth}}{{1pt}} % Header rule thickness
    \\renewcommand{{\\footrulewidth}}{{1pt}} % Footer rule thickness
}}

\\input{{variables.tex}}

\\AtBeginDocument{{\\thispagestyle{{firstpage}}}} % Use the first page headers/footers style on the first page
\\pagestyle{{subsequentpages}} % Use the subsequent pages headers/footers style on subsequent pages
%----------------------------------------------------------------------------------------
%----------------------------------------------------------------------------------------
\\begin{{document}}
\\rule{{\\linewidth}}{{1pt}} % Horizontal rule

% Use the commands in your document
\\begin{{center}}
    Jannik M. B. Sørensen |
    Email: Mangabat93@gmail.com | 
    Odder, Denmark | Date
\\end{{center}}

\\subsection*{{\\finalJobtitle, \\finalCompanyName}}
    \\begin{{spacing}}{{1.2}}
        {{\\finalIntroduction}}
    \\end{{spacing}}
    \\vspace*{{0mm}}

\\subsection*{{Motivation}}
    \\begin{{spacing}}{{1.2}}
        {{\\finalMotivation}}
    \\end{{spacing}}
    \\vspace*{{0mm}}

\\subsection*{{Skills}}
    \\begin{{spacing}}{{1.2}}
        {{\\finalSkills}}
    \\end{{spacing}}
    \\vspace*{{0mm}}

\\subsection*{{Education}}
% Masters degree
    \\begin{{spacing}}{{1.2}}
        {{\\finalEducationMaster}}\\\\
        % Bachelors degree
        {{\\finalEducationBachelor}}
    \\end{{spacing}}
    \\vspace*{{0mm}}

\\subsection*{{Continued Learning}}
    \\begin{{spacing}}{{1.2}}
        {{\\finalContinuedLearning}}
    \\end{{spacing}}
    \\vspace*{{0mm}}

\\subsection*{{Thanks for your time}}
    \\begin{{spacing}}{{1.2}}
        {{\\finalThankYou}}\\\\        
    \\end{{spacing}}

\\noindent Kind regards,\\\\
    Jannik Mangabat
%----------------------------------------------------------------------------------------
% LETTER CONTENT
%----------------------------------------------------------------------------------------

\\end{{document}}
"""

with open(latex_file_path, 'w') as file:
    file.write(tex_content)

Number of safe texts: 4
Number of modified texts: 5
